In [1]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from snakemake_stub import *

from scipy.sparse import csr_matrix

import gzip, json, collections
from typing import Sequence, Mapping, Collection
from Bio import SeqIO
import scipy as sp
import numpy as np
import pandas as pd
import sys
sys.path.append("scripts")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
def init_reverse_complement():
    TRANSLATION_TABLE = str.maketrans("ACTGactg", "TGACtgac")

    def reverse_complement(sequence: str) -> str:
        """
        >>> reverse_complement("AATC")
        'GATT'
        >>> reverse_complement("CCANT")
        'ANTGG'
        """
        sequence = str(sequence)
        return sequence.translate(TRANSLATION_TABLE)[::-1]

    return reverse_complement


reverse_complement = init_reverse_complement()
import mmh3
import sharedmem
from sklearn.neighbors import NearestNeighbors  
import numpy as np  



In [2]:
import numpy as np
from numpy.typing import NDArray
import anndata

class _SpectralMatrixFree:
    """
    Perform dimension reduction using Laplacian Eigenmaps.

    Matrix-free spectral embedding without computing the similarity matrix explicitly.

    Only cosine similarity is supported.

    Adapted from https://github.com/kaizhang/SnapATAC2/blob/51f040c095820fea43e9a6360d751bfc29faecc5/snapatac2-python/python/snapatac2/tools/_embedding.py#L434
    """

    def __init__(
        self,
        out_dim: int = 30,
        feature_weights=None,
    ):
        self.out_dim = out_dim
        self.feature_weights = feature_weights

    def fit(self, mat):
        if self.feature_weights is not None:
            mat = mat @ sp.sparse.diags(self.feature_weights)
        self.sample = mat
        self.in_dim = mat.shape[1]

        s = 1 / np.sqrt(np.ravel(sp.sparse.csr_matrix.power(mat, 2).sum(axis=1)))
        X = sp.sparse.diags(s) @ mat

        D = np.ravel(X @ X.sum(axis=0).T) - 1
        X = sp.sparse.diags(1 / np.sqrt(D)) @ X
        evals, evecs = self._eigen(X, 1 / D, k=self.out_dim)

        ix = evals.argsort()[::-1]
        self.evals = evals[ix]
        self.evecs = evecs[:, ix]

    def transform(self, weighted_by_sd: bool = True):
        evals = self.evals
        evecs = self.evecs

        if weighted_by_sd:
            idx = [i for i in range(evals.shape[0]) if evals[i] > 0]
            evals = evals[idx]
            evecs = evecs[:, idx] * np.sqrt(evals)
        return evals, evecs

    @staticmethod
    def _eigen(X, D, k):
        def f(v):
            return X @ (v.T @ X).T - D * v

        n = X.shape[0]
        A = sp.sparse.linalg.LinearOperator((n, n), matvec=f, dtype=np.float64)
        return sp.sparse.linalg.eigsh(A, k=k)

class _DimensionReduction:

    def transform(self, data: csr_matrix | NDArray, n_dimensions: int) -> NDArray:
        raise NotImplementedError


class SpectralEmbedding(_DimensionReduction):
    def transform(
        self, data: csr_matrix | NDArray, n_dimensions: int, weighted_by_sd: bool = True
    ) -> NDArray:
        reducer = _SpectralMatrixFree(out_dim=n_dimensions)
        reducer.fit(data)
        _, embedding = reducer.transform(weighted_by_sd=weighted_by_sd)
        return embedding

In [ ]:
x = np.load('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/metagenome/pbsim_ONT_95_20k/kmer_16/HNSW_Cosine_Spectural_500d_IDF_nbr_matrix.npz')

In [6]:
x['arr_0'][300]

array([81663], dtype=uint64)

In [34]:
all_ref_database = '/home/miaocj/docker_dir/kNN-overlap-finder/data/metagenome_reference/GCR.fa' ## include 1310 species 
origin_ref_datebase = '/home/miaocj/docker_dir/kNN-overlap-finder/data/metagenome_reference/GCR.fa.split/GCR.part_001.fa' 
##used for generate simulated reads, include 19 species
query_reads = '/home/miaocj/docker_dir/kNN-overlap-finder/data/metagenome_reads/part001/pbsim_ONT_95_20k/reads.fa'

##Part1: generate a read-species dict, using for final examination
all_ref_reads_tax_list = []
with open(all_ref_database, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        tax=record.id[:6]
        all_ref_reads_tax_list.append(tax)##ID example: QSQV01000067.1, 'QSQV01' present species code
all_ref_read_tax = {i:tax for i,tax in enumerate(all_ref_reads_tax_list)}


ref_reads_tax_list = []
with open(origin_ref_datebase, "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        tax=record.id[:6]
        ref_reads_tax_list.append(tax)##ID example: QSQV01000067.1, 'QSQV01' present species code
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1
que_read_tax_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/que_read_tax.json'
ref_read_tax_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/ref_read_tax.json'

with gzip.open(que_read_tax_file, "wt") as f:
    json.dump(que_read_tax, f)
with gzip.open(ref_read_tax_file, "wt") as f:
    json.dump(all_ref_read_tax, f)

In [3]:
ref_read_tax_file ='/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/ref_read_tax.json.gz'
que_read_tax_file ='/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/que_read_tax.json.gz'
ref_feature_matrix_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/que_feature_matrix.npz'
que_feature_matrix_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/ref_feature_matrix.npz'

with gzip.open(ref_read_tax_file, "rt") as f:
    ref_read_tax = json.load(f)
with gzip.open(que_read_tax_file, "rt") as f:
    que_read_tax = json.load(f)

ref_feature_matrix = sp.sparse.load_npz(ref_feature_matrix_file)
que_feature_matrix = sp.sparse.load_npz(que_feature_matrix_file)

In [ ]:
from sklearn.preprocessing import normalize as normalize_function

class scBiMapEmbedding(_DimensionReduction):
    """
    From scBiMapping on Conda
    Author: Teng Qiu
    """

    def transform(
        self,
        data: csr_matrix | NDArray,
        n_dimensions: int,
        *,
        normalize=True,
    ) -> NDArray:
        if data.min() < 0:
            raise ValueError(
                "The input matrix is regarded as a similarity matrix and thus should not contain negtive values"
            )
        eps=0.0000000001,
        Dx = sp.sparse.diags(np.ravel(1 / (data.sum(axis=1) + eps)))

        y = (Dx @ csr_matrix(data.sum(axis=1))).T  # row vector
        Dy = sp.sparse.diags(np.ravel((1 / (np.sqrt((y @ data).T.toarray()) + eps))))  # type: ignore
        C = np.sqrt(Dx) @ data @ Dy  # type: ignore
        _, _, evec = sp.sparse.linalg.svds( # type: ignore
            C, k=n_dimensions, return_singular_vectors="vh", random_state=0,
        )
        V = Dy @ evec.T  # eigenvectors for features
        U = Dx @ data @ V  # eigenvectors for cells

        if normalize:
            U = normalize_function(U, axis=1, norm="l2")
        return U  # type: ignore

In [21]:
sp.__version__

'1.14.0'

In [16]:
sp.__file__

'/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/3172702e82c9c1d9450fdd20452651b9_/lib/python3.12/site-packages/scipy/__init__.py'

In [18]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import svds
mat = csc_array((65536, 65536))
_, s, _ = svds(mat)
print(s[0])

0.0


In [ ]:
from math import ceil
from typing import Sequence, Type, Mapping, Iterable, Literal

class LowHash():

    @staticmethod
    def _hash(x: int, seed: int) -> int:
        hash_value = mmh3.hash(str(x), seed=int(seed))
        return hash_value

    @staticmethod
    def _get_hash_values(data: Iterable[int], repeats: int, seed: int) -> np.ndarray:
        rng = np.random.default_rng(seed)
        hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeats, dtype=np.uint64)
        hash_values = []
        for k in range(repeats):
            s = hash_seeds[k]
            for x in data:
                hash_values.append(LowHash._hash(x, seed=s))
        hash_values = np.array(hash_values, dtype=np.int64)
        return hash_values

    @staticmethod
    def _get_lowhash_count(
        hash_count: int,
        lowhash_fraction: float | None = None,
        lowhash_count: int | None = None,
    ) -> int:
        if lowhash_fraction is None and lowhash_count is None:
            raise TypeError(
                "Either `lowhash_fraction` or `lowhash_count` must be specified."
            )
        if lowhash_fraction is not None and lowhash_count is not None:
            raise TypeError(
                f"`lowhash_fraction` and `lowhash_count` cannot be specified at the same time. {lowhash_fraction=} {lowhash_count=}"
            )

        if lowhash_fraction is not None:
            lowhash_count = ceil(hash_count * lowhash_fraction)
            lowhash_count = max(lowhash_count, 1)
        if lowhash_count is None:
            raise ValueError()
        return lowhash_count

    def _lowhash(
        
        data: csr_matrix | np.ndarray,
        repeats: int,
        lowhash_fraction: float | None,
        lowhash_count: int | None = None,
        seed: int = 5731343,
        verbose=True,
    ) -> csr_matrix:
        sample_count, feature_count = data.shape
        buckets = sp.sparse.dok_matrix(
            (feature_count * repeats, sample_count), dtype=np.bool_
        )

        # Calculate hash values
        hash_values = self._get_hash_values(
            np.arange(feature_count), repeats=repeats, seed=seed
        )

        # For each sample, find the lowest hash values for its features
        for j in range(sample_count):
            feature_indices = sp.sparse.find(data[j, :] > 0)[1]
            hash_count = feature_indices.shape[0]
            sample_lowhash_count = self._get_lowhash_count(
                hash_count=hash_count,
                lowhash_fraction=lowhash_fraction,
                lowhash_count=lowhash_count,
            )
            for k in range(repeats):
                bucket_indices = feature_indices + (k * feature_count)
                sample_hash_values = hash_values[bucket_indices]
                low_hash_buckets = bucket_indices[
                    np.argsort(sample_hash_values)[:sample_lowhash_count]
                ]
                buckets[low_hash_buckets, j] = 1
            if verbose and j % 1000 == 0:
                print(j, end=" ")
        if verbose:
            print("")
        buckets = sp.sparse.csr_matrix(buckets)
        return buckets


In [9]:
data = sp.sparse.vstack([ref_feature_matrix,que_feature_matrix])
buckets = LowHash()._lowhash(data,repeats=100,lowhash_fraction=0.01,lowhash_count=None)

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 101000 102000 103000 104000 105000 106000 107000 108000 109000 110000 111000 112000 113000 114000 115000 116000 117000 118000 119000 120000 121000 122000 123000 124000 125000 126000 127000 128000 129000 130000 131000 132000 133000 134000 135000 136000 137000 138000 139000 140000 141000 142000 143000 144000 145000 146000 147000 148000 149000 150000 151000 152000 153000 154000 155000 156000 157000 158000 

In [12]:
row_sums = buckets.sum(axis=1).A1  # type: ignore
matrix = buckets[
    (row_sums >= 2) & (row_sums <= float("inf")), :
].astype(np.uint8)

In [13]:
matrix.shape

(15396476, 332922)

In [ ]:
buckets.shape 

(1078533000, 332922)

In [11]:
data.shape

(332922, 10785330)

In [20]:
data = sp.sparse.vstack([ref_feature_matrix,que_feature_matrix])
ref_read_num = ref_feature_matrix.shape[0]
_data = scBiMapEmbedding().transform(data,n_dimensions=500)

TypeError: svds() got an unexpected keyword argument 'lapack_driver'

In [ ]:
eps=0.0000000001,
Dx = sp.sparse.diags(np.ravel(1 / (data.sum(axis=1) + eps)))

y = (Dx @ csr_matrix(data.sum(axis=1))).T  # row vector
Dy = sp.sparse.diags(np.ravel((1 / (np.sqrt((y @ data).T.toarray()) + eps))))  # type: ignore
C = np.sqrt(Dx) @ data @ Dy  # type: ignore

In [ ]:
_ref = _data[:ref_read_num]
_que = _data[ref_read_num:]


In [20]:
indice = x['arr_0']

In [46]:
len(indice)

80256

In [48]:
indice.max()

250507

In [52]:
actual = []
prediction = []
for query_read_num,x in enumerate(indice):
    neighbor = x[0]
    neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
    query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
    prediction.append(ref_read_tax[str(int(neighbor))])
    actual.append(que_read_tax[str(int(query_read_num))])

In [ ]:
precision_sep = precision_score(actual, prediction, average=None)  

/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/3172702e82c9c1d9450fdd20452651b9_/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
mypre=[]
mysen=[]
for i in set(actual):
    tp = 0
    fp = 0
    fn = 0
    for x in range(len(actual)):
        if actual[x] == i:
            if actual[x] == prediction[x]:
                tp += 1
            else:
                fn += 1
        else:
            if prediction[x] == i:
                fp += 1
    pre = tp/(tp+fp)
    sen = tp/(tp+fn)
    mypre.append(pre)
    mysen.append(sen)

In [91]:
sum(mypre)/len(mypre)

0.8651967756731813

In [80]:
len(set(prediction))

353

In [83]:
len(set(precision_sep))

17

In [77]:
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indice,ref_read_tax,que_read_tax)

/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/3172702e82c9c1d9450fdd20452651b9_/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/3172702e82c9c1d9450fdd20452651b9_/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [78]:
precision

0.04656866497957633

In [50]:
len(precision_sep)

353

In [20]:
##calculate sensitivity and precision

from sklearn.metrics import precision_score, recall_score  

def evaluate(indices,ref_read_tax,que_read_tax):
    actual = []
    prediction = []
    for query_read_num,x in enumerate(indices):
        neighbor = x[0]
        neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
        query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
        prediction.append(ref_read_tax[str(int(neighbor))])
        actual.append(que_read_tax[str(int(query_read_num))])

    precision = precision_score(actual, prediction,average='macro')
    sensitivity = recall_score(actual, prediction,average='macro')
    ##计算每个类别的
    precision_sep = precision_score(actual, prediction, average=None)  
    sensitivity_sep = recall_score(actual, prediction, average=None)
    return precision,sensitivity,precision_sep,sensitivity_sep

In [2]:
ref_database = '/home/miaocj/docker_dir/data/metagenome/part1.fa'
query_reads = '/home/miaocj/docker_dir/data/metagenome/pbsim_ONT_95_30k_10dep_part1_reads.fasta'

In [3]:
ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            start = lines.index('S')
            end = lines.index('_')
            ref_num = lines[start+1:end]
            que_read_tax[flag] = ref_read_tax[int(ref_num)-1]
            flag +=1


In [7]:
que_read_tax_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/bacteria/que_read_tax.json'
ref_read_tax_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/bacteria/ref_read_tax.json'

with gzip.open(que_read_tax_file, "wt") as f:
    json.dump(que_read_tax, f)
with gzip.open(ref_read_tax_file, "wt") as f:
    json.dump(ref_read_tax, f)

In [8]:
def load_reads(fasta_path: str):
    read_sequences = []
    read_names = []
    read_orientations = []

    with open(fasta_path, "rt") as handle:  # Open gzipped file in text mode
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq)
            read_sequences.append(seq)
            read_names.append(record.id)
            read_orientations.append("+")

            # Include reverse complement
            read_sequences.append(reverse_complement(seq))
            read_names.append(record.id)
            read_orientations.append("-")

    return read_names, read_orientations, read_sequences

read_names, read_orientations, read_sequences = load_reads(ref_database)


In [5]:
def build_kmer_index(
    read_sequences: Sequence[str],
    k: int,
    *,
    sample_fraction: float,
    min_multiplicity: int,
    seed: int,
) -> Mapping[str, int]:
    kmer_counter = collections.Counter()
    for seq in read_sequences:
        for p in range(len(seq) - k + 1):
            kmer = seq[p : p + k]
            kmer_counter[kmer] += 1

    kmer_spectrum = collections.Counter(x for x in kmer_counter.values() if x <= 10)
    print(kmer_spectrum)

    rng = np.random.default_rng(seed=seed)
    vocabulary = set(
        x
        for x, count in kmer_counter.items()
        if count >= min_multiplicity and rng.random() <= sample_fraction
    )
    vocabulary |= set(reverse_complement(x) for x in vocabulary)
    kmer_indices = {kmer: i for i, kmer in enumerate(vocabulary)}
    return kmer_indices

sample_fraction=0.1
min_multiplicity=2
seed=562104830
kmer_indices = build_kmer_index(        
        read_sequences=read_sequences,
        k=16,
        sample_fraction=sample_fraction,
        min_multiplicity=min_multiplicity,
        seed=seed)


Counter({1: 276256424, 2: 23715451, 3: 3702956, 4: 1039535, 5: 423488, 6: 221958, 7: 134826, 8: 86799, 9: 59680, 10: 48269})


In [6]:
qread_names, qread_orientations, qread_sequences = load_reads(query_reads)

In [8]:
def build_feature_matrix(
    read_sequences: Sequence[str],
    kmer_indices: Mapping[str, int],
    k: int,
) -> tuple[sp.csr_matrix, Sequence[Sequence[int]]]:
    row_ind, col_ind, data = [], [], []
    read_features = []
    for i, seq in enumerate(read_sequences):
        features_i = []
        for p in range(len(seq) - k + 1):
            kmer = seq[p : p + k]
            j = kmer_indices.get(kmer)
            if j is None:
                continue
            features_i.append(j)

        read_features.append(features_i)

        kmer_counts = collections.Counter(features_i)
        for j, count in kmer_counts.items():
            row_ind.append(i)
            col_ind.append(j)
            data.append(count)

    feature_matrix = sp.csr_matrix(
        (data, (row_ind, col_ind)), shape=(len(read_sequences), len(kmer_indices))
    )
    return feature_matrix, read_features

In [9]:
ref_feature_matrix,ref_read_features = build_feature_matrix(read_sequences=read_sequences,kmer_indices=kmer_indices,k=16)

In [11]:
ref_feature_matrix.shape

(1000, 5647520)

In [15]:
from isal import igzip
def open_gzipped(path, mode="rt", gzipped: bool | None = None, **kw):
    if gzipped is None:
        gzipped = path.endswith(".gz")
    if gzipped:
        open_ = igzip.open
        return open_(path, mode)
    else:
        open_ = open
    return open_(path, mode, **kw)
                   
output_npz_file = '/home/miaocj/docker_dir/data/metagenome/ref_feature_matrix.npz'
output_json_file = '/home/miaocj/docker_dir/data/metagenome/ref_read_features.json.gz'
sp.save_npz(output_npz_file, ref_feature_matrix)
with open_gzipped(output_json_file, "wt") as f:
    json.dump(ref_read_features, f)

In [10]:
que_feature_matrix,que_read_features = build_feature_matrix(read_sequences=qread_sequences,kmer_indices=kmer_indices,k=16)

In [16]:
##存储
output_npz_file2 = '/home/miaocj/docker_dir/data/metagenome/que_feature_matrix.npz'
output_json_file2 = '/home/miaocj/docker_dir/data/metagenome/que_read_features.json.gz'
sp.save_npz(output_npz_file2, que_feature_matrix)
with open_gzipped(output_json_file2, "wt") as f:
    json.dump(que_read_features, f)

In [3]:
output_ref_npz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/ref_feature_matrix.npz'
output_que_npz_file= '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/que_feature_matrix.npz'
output_ref_json_file= '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/ref_read_features.json.gz.npz'
output_que_json_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/pbsim_ONT_95_20k/kmer_16/que_read_features.json.gz'

# with gzip.open(output_json_file2, "rt") as f:
#     que_read_features = json.load(f)
ref_feature_matrix = sp.load_npz(output_ref_npz_file)
que_feature_matrix = sp.load_npz(output_que_npz_file)


In [10]:
##读取
output_npz_file2 = '/home/miaocj/docker_dir/data/metagenome/que_feature_matrix.npz'
output_npz_file = '/home/miaocj/docker_dir/data/metagenome/ref_feature_matrix.npz'
output_json_file2 = '/home/miaocj/docker_dir/data/metagenome/que_read_features.json.gz'
ref_feature_matrix = sp.sparse.load_npz(output_npz_file)
que_feature_matrix = sp.sparse.load_npz(output_npz_file2)

In [11]:
ref_feature_matrix.shape

(1000, 2899125)

In [12]:
que_feature_matrix.shape

(119250, 2899125)

In [19]:
data =sp.vstack([ref_feature_matrix, que_feature_matrix])  

In [20]:
data.shape

(332922, 10785330)

In [ ]:
ref_read_num = ref_feature_matrix.shape[0]

_ref = data[:ref_read_num]
_que = data[ref_read_num:]

In [ ]:
_ref

(2, 1)

In [6]:
ref_feature_matrix.shape

(252666, 10785330)

In [13]:
from scipy import sparse
from scipy.sparse._csr import csr_matrix
import numpy as np
from numpy import matlib, ndarray
from numpy.typing import NDArray
import sklearn.neighbors

class _NearestNeighbors:
    def get_neighbors(
        self, ref: csr_matrix | np.ndarray, que: csr_matrix | np.ndarray, n_neighbors: int
    ) -> np.ndarray:
        raise NotImplementedError()

class ExactNearestNeighbors(_NearestNeighbors):
    def get_neighbors(
        self, ref: csr_matrix | np.ndarray, que: csr_matrix | np.ndarray, metric="cosine", n_neighbors: int = 20
    ):
        
        nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=1, algorithm='auto',metric=metric).fit(ref)  
        _, nbr_indices = nbrs.kneighbors(que)
        return nbr_indices


In [ ]:
exact=ExactNearestNeighbors()
nbr=exact.get_neighbors(ref_feature_matrix,que_feature_matrix)

In [32]:
ref_feature_matrix1= sp.load_npz(output_ref_npz_file)

In [33]:
ref_feature_matrix1.shape

(252666, 10785330)

In [27]:
ref_feature_matrix['shape']

array([  252666, 10785330])

In [30]:
que_feature_matrix['shape']

array([   80256, 10785330])

In [24]:
print(que_feature_matrix.keys())
sparse_data = sp.csr_matrix(())


KeysView(NpzFile '/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/metagenome/ref_feature_matrix.npz' with keys: indices, indptr, format, shape, data)


In [25]:
sparse_data.shape

(1, 12485670)

In [15]:
##读取
output_npz_file2 = '/home/miaocj/docker_dir/data/metagenome/que_feature_matrix.npz'
output_json_file2 = '/home/miaocj/docker_dir/data/metagenome/que_read_features.json.gz'
with gzip.open(output_json_file2, "rt") as f:
    que_read_features = json.load(f)
que_feature_matrix = sp.sparse.load_npz(output_npz_file2)

output_npz_file = '/home/miaocj/docker_dir/data/metagenome/ref_feature_matrix.npz'
output_json_file = '/home/miaocj/docker_dir/data/metagenome/ref_read_features.json.gz'
with gzip.open(output_json_file, "rt") as f:
    ref_read_features = json.load(f)
ref_feature_matrix = sp.sparse.load_npz(output_npz_file)

In [13]:
que_feature_matrix.shape

(119250, 5647520)

In [ ]:
empty_indices = [index for index, sublist in enumerate(que_read_features) if len(sublist) == 0]  
empty_indices

In [19]:
sys.path.append("scripts")
sys.path.append("../../../scripts")
merged_matrix = sp.sparse.vstack([ref_feature_matrix, que_feature_matrix])  
from dim_reduction import scBiMapEmbedding
dim500 = scBiMapEmbedding().transform(merged_matrix,n_dimensions=500)

/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/3172702e82c9c1d9450fdd20452651b9_/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-12 06:21:10.226389: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 06:21:11.465526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with 

In [14]:
dim500_ref = dim500[:1000]
dim500_que = dim500[1001:]

In [15]:
## 降维+精确求解

query_vectors = np.array(dim500_que) 
database_vectors = np.array(dim500_ref) 
nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto',metric='cosine').fit(database_vectors)  
distances, indices = nbrs.kneighbors(query_vectors)
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices)

In [16]:
precision,sensitivity

(0.970391944806762, 0.9575761776138907)

In [ ]:
import matplotlib.pyplot as plt
categories = []
for i in range(1,13):
    categories.append(i)

plt.figure(figsize=(4,3), dpi=300) 
plt.bar(categories,precision_sep,color = 'gray')

plt.ylabel('Genus Precision')
plt.ylim(0.6,1.01)
plt.xticks(rotation=45, ha='right') 
plt.gca().spines['right'].set_color('none')  
plt.gca().spines['top'].set_color('none') 

In [ ]:
import matplotlib.pyplot as plt
import seaborn
plt.figure(figsize=(6, 6), dpi=300) 
plt.scatter(precision_sep,sensitivity_sep, label='Data points', marker='o', c=sensitivity_sep, cmap='rainbow')

plt.xlim(0.8,1.01)
plt.ylim(0.8,1.01)
plt.gca().spines['right'].set_color('none')  
plt.gca().spines['top'].set_color('none') 
plt.xlabel('precision')
plt.ylabel('sensitivity')
plt.margins(0.3) 

In [114]:
##rough evaluate
right_num = 0
for query_read_num,x in enumerate(indices):
    neighbor = x[0]
    neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
    query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
    if ref_read_tax[neighbor] == que_read_tax[query_read_num]:
        right_num +=1

print(right_num/len(indices))

0.9584399030599837


In [65]:
precision_sep,sensitivity_sep

(array([0.87046535, 0.62569832, 0.83373867, 0.86720252, 0.89721499,
        0.87860642, 0.86374657, 0.41751232, 0.90688083, 0.54479376,
        0.83079625, 0.93578767]),
 array([0.47510944, 0.98778833, 0.61177807, 0.68116937, 0.66164303,
        0.6721266 , 0.60990641, 0.99246462, 0.41897312, 0.68407741,
        0.71957404, 0.33051104]))

In [41]:
from dataclasses import dataclass, field
from functools import lru_cache
from typing import Sequence, Type, Mapping, Iterable, Literal
from warnings import warn
from math import ceil
from scipy.sparse._csr import csr_matrix
import numpy as np
import hnswlib

class HNSW():
    def get_neighbors(
        self,
        ref_data: csr_matrix | np.ndarray,
        que_data: csr_matrix | np.ndarray,
        n_neighbors: int,
        metric: Literal["euclidean", "cosine"] = "euclidean",
        *,
        threads: int | None = None,
        M: int = 16,
        ef_construction: int = 200,
        ef_search: int = 50,
    ) -> np.ndarray:
        if metric == "euclidean":
            space = "l2"
        else:
            space = metric

        # Initialize the HNSW index
        p = hnswlib.Index(space=space, dim=ref_data.shape[1])
        if threads is not None:
            p.set_num_threads(threads)
        p.init_index(max_elements=ref_data.shape[0], ef_construction=ef_construction, M=M)
        ids = np.arange(ref_data.shape[0])
        p.add_items(ref_data, ids)
        p.set_ef(ef_search)
        nbr_indices, _ = p.knn_query(que_data, k=n_neighbors)
        return nbr_indices
nbr_indices = HNSW().get_neighbors(ref_data=dim500_ref,que_data=dim500_que,n_neighbors=1)


In [42]:
##dim500_HNSW

indices = HNSW().get_neighbors(ref_data=dim500_ref,que_data=dim500_que,n_neighbors=1)
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices)

In [46]:
precision,sensitivity

(0.9581952089324665, 0.952445601680645)

In [46]:
from dataclasses import dataclass, field
from functools import lru_cache
import collections
from typing import Sequence, Type, Mapping, Iterable, Literal
from warnings import warn
from math import ceil
from scipy import sparse
from scipy.sparse._csr import csr_matrix
import numpy as np
import mmh3




class LowHash():

    @staticmethod
    def _hash(x: int, seed: int) -> int:
        hash_value = mmh3.hash(str(x), seed=seed)
        return hash_value

    @staticmethod
    def _get_hash_values(data: Iterable[int], repeats: int, seed: int) -> np.ndarray:
        rng = np.random.default_rng(seed)
        hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeats, dtype=np.uint64)
        hash_values = []
        for k in range(repeats):
            s = hash_seeds[k]
            for x in data:
                hash_values.append(LowHash._hash(x, seed=s))
        hash_values = np.array(hash_values, dtype=np.int64)
        return hash_values

    @staticmethod
    def _get_lowhash_count(
        hash_count: int,
        lowhash_fraction: float | None = None,
        lowhash_count: int | None = None,
    ) -> int:
        if lowhash_fraction is None and lowhash_count is None:
            raise TypeError(
                "Either `lowhash_fraction` or `lowhash_count` must be specified."
            )
        if lowhash_fraction is not None and lowhash_count is not None:
            raise TypeError(
                f"`lowhash_fraction` and `lowhash_count` cannot be specified at the same time. {lowhash_fraction=} {lowhash_count=}"
            )

        if lowhash_fraction is not None:
            lowhash_count = ceil(hash_count * lowhash_fraction)
            lowhash_count = max(lowhash_count, 1)
        if lowhash_count is None:
            raise ValueError()
        return lowhash_count
    
    def _lowhash(
        self,
        data: csr_matrix | np.ndarray,
        repeats: int,
        lowhash_fraction: float | None,
        lowhash_count: int | None = None,
        seed: int = 5731343,
        verbose=True,
    ) -> csr_matrix:
        sample_count, feature_count = data.shape
        buckets = sparse.dok_matrix(
            (feature_count * repeats, sample_count), dtype=np.bool_
        )

        # Calculate hash values
        hash_values = self._get_hash_values(
            np.arange(feature_count), repeats=repeats, seed=seed
        )

        # For each sample, find the lowest hash values for its features
        for j in range(sample_count):
            feature_indices = sparse.find(data[j, :] > 0)[1]
            hash_count = feature_indices.shape[0]
            sample_lowhash_count = self._get_lowhash_count(
                hash_count=hash_count,
                lowhash_fraction=lowhash_fraction,
                lowhash_count=lowhash_count,
            )
            for k in range(repeats):
                bucket_indices = feature_indices + (k * feature_count)
                sample_hash_values = hash_values[bucket_indices]
                low_hash_buckets = bucket_indices[
                    np.argsort(sample_hash_values)[:sample_lowhash_count]
                ]
                buckets[low_hash_buckets, j] = 1
            if verbose and j % 1000 == 0:
                print(j, end=" ")
        if verbose:
            print("")
        buckets = sparse.csr_matrix(buckets)
        return buckets

    def _get_adjacency_matrix(
        self,
        data: csr_matrix | np.ndarray,
        buckets: csr_matrix,
        n_neighbors: int,
        min_bucket_size,
        max_bucket_size,
        min_cooccurence_count,
    ) -> np.ndarray:

        # Select neighbor candidates based on cooccurence counts
        row_sums = buckets.sum(axis=1).A1  # type: ignore
        matrix = buckets[
            (row_sums >= min_bucket_size) & (row_sums <= max_bucket_size), :
        ].astype(np.uint8)
        cooccurrence_matrix = matrix.T.dot(matrix)

        neighbor_dict = collections.defaultdict(dict)
        nonzero_indices = list(zip(*cooccurrence_matrix.nonzero()))
        for i, j in nonzero_indices:
            if i >= j: 
                continue

            count = cooccurrence_matrix[i, j]
            neighbor_dict[i][j] = count
            neighbor_dict[j][i] = count

        # Construct neighbor matrix
        n_rows = data.shape[0]
        nbr_matrix = []
        for i in range(n_rows)[1000:]:
            row_nbr_dict = {
                j: count
                for j, count in neighbor_dict[i].items()
                if count >= min_cooccurence_count and j < 1000
            }
            neighbors = list(
                sorted(row_nbr_dict, key=lambda x: row_nbr_dict[x], reverse=True)
            )[:n_neighbors]
            nbr_matrix.append(neighbors)
        return nbr_matrix

    def get_neighbors(
        self,
        data: csr_matrix | np.ndarray,
        n_neighbors: int,
        lowhash_fraction: float | None = None,
        lowhash_count: int | None = None,
        repeats=100,
        min_bucket_size=2,
        max_bucket_size=float("inf"),
        min_cooccurence_count=1,
        *,
        seed=1,
        verbose=True,
    ) -> np.ndarray:

        buckets = self._lowhash(
            data,
            repeats=repeats,
            lowhash_fraction=lowhash_fraction,
            lowhash_count=lowhash_count,
            seed=seed,
            verbose=verbose,
        )
        nbr_matrix = self._get_adjacency_matrix(
            data,
            buckets,
            n_neighbors=n_neighbors,
            min_bucket_size=min_bucket_size,
            max_bucket_size=max_bucket_size,
            min_cooccurence_count=min_cooccurence_count,
        )
        return nbr_matrix


In [ ]:
COVERAGE_DEPTH = 20
max_bucket_size = COVERAGE_DEPTH * 1.5
indices = LowHash().get_neighbors(data=merged_matrix,repeats=100,lowhash_count=20,n_neighbors=1,
            max_bucket_size=max_bucket_size,
            seed=458)

In [ ]:
merged_matrix.shape